In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [28]:
url_inicial = 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/archive/3'
url_root = 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/archive/3'
r=requests.get(url_inicial)

In [29]:
soup = BeautifulSoup(r.text, 'html.parser')

In [30]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/14',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/13']

In [31]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('ul', class_='cmp_article_list articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [32]:
vol2

['https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/156',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/145']

In [33]:
def get_url_items(sopa,url):
    box1 = soup1.find('ul', class_='cmp_article_list articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [34]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/archive/3


In [35]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

2

In [36]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [37]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Urología'
    a='III. Medicina y Ciencias de la Salud'
    tem='Urología'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('section', class_='item abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [38]:
list_scraper=list_scraper[0:77]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1


In [39]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Cáncer de próstata y bisfenoles,None,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
1,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,De mingitorios y esas cosas,None,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...


In [40]:
df_catalogo.to_csv('Revista93.03.csv', index=False)